In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import pandas as pd
from tqdm import tqdm
from PFNExperiments.Evaluation.ClassifcationBasedComparison import compare_samples_classifier_based
from PFNExperiments.Evaluation.MMD import compare_samples_mmd
from scipy.stats import wilcoxon

In [3]:
# sample data such that the marginals for x and y are always the same, only the joint distribution changes

P = 1
N_samples = 1000

def samplex(N_samples = N_samples):
    return torch.randn(N_samples, P)

def generate_sample_y_t(t):
    def sampley_t(x):
        """
        Interpolate between sampley3 and sampley1
        t = 1 -> deterministic dependence on x
        t = 0 -> no dependence on x
        """
        xt = 2*t*x

        noise = torch.randn(N_samples, P)*((4 - (2*t)**2)**0.5)

        return xt + noise
    return sampley_t



sampley_gt = generate_sample_y_t(1)
t_max = 0.99

In [5]:
t_max = 0.99
sampley1 = generate_sample_y_t(t_max)

In [6]:
# consistency check if any difference in the marginals regarding y can be detected

def sample_marginal(n_x_samples, sample_ya, sample_yb):
    res_list = []
    for i in tqdm(list(range(n_x_samples))):
        x1 = samplex()
        x2 = samplex()
        y1 = sample_ya(x1)
        y2 = sample_yb(x2)
        diff_mmd = compare_samples_classifier_based(y1, y2, n_folds=3)

        res_list.append(diff_mmd)

    return pd.DataFrame(res_list)

marginal_y1y2_res = sample_marginal(n_x_samples = 100, sample_ya = sampley1, sample_yb = generate_sample_y_t(0.5))
marginal_y1y3_res = sample_marginal(n_x_samples = 100, sample_ya = sampley1, sample_yb = generate_sample_y_t(0.0))

print(f"""
      marginal_y1y2_res: {marginal_y1y2_res.mean()}
        marginal_y1y3_res: {marginal_y1y3_res.mean()}
        """)

print(wilcoxon(marginal_y1y2_res, marginal_y1y3_res))

100%|██████████| 100/100 [01:11<00:00,  1.39it/s]


      marginal_y1y2_res: CST_accuracy         0.495813
CST_roc_auc_score    0.495844
dtype: float64
        marginal_y1y3_res: CST_accuracy         0.504255
CST_roc_auc_score    0.504296
dtype: float64
        
WilcoxonResult(statistic=array([2120., 2113.]), pvalue=array([0.16376327, 0.15660166]))


In [7]:
different_t_res = []
for t in tqdm(torch.linspace(0, t_max, 10)):
    marginal_y1yt_res = sample_marginal(n_x_samples = 100, sample_ya = sampley1, sample_yb = generate_sample_y_t(t))
    different_t_res.append(marginal_y1yt_res.mean())

print(different_t_res)

100%|██████████| 10/10 [48:25<00:00, 290.58s/it]

[CST_accuracy         0.491727
CST_roc_auc_score    0.491760
dtype: float64, CST_accuracy         0.506863
CST_roc_auc_score    0.506919
dtype: float64, CST_accuracy         0.502577
CST_roc_auc_score    0.502617
dtype: float64, CST_accuracy         0.494149
CST_roc_auc_score    0.494214
dtype: float64, CST_accuracy         0.499879
CST_roc_auc_score    0.499929
dtype: float64, CST_accuracy         0.498941
CST_roc_auc_score    0.499018
dtype: float64, CST_accuracy         0.501841
CST_roc_auc_score    0.501903
dtype: float64, CST_accuracy         0.500224
CST_roc_auc_score    0.500403
dtype: float64, CST_accuracy         0.498377
CST_roc_auc_score    0.498454
dtype: float64, CST_accuracy         0.501729
CST_roc_auc_score    0.501846
dtype: float64]


In [8]:
# check expected difference in the posteriors


def sample_posterior(n_x_samples, sample_ya, sample_yb):
    res_list = []
    for i in tqdm(list(range(n_x_samples))):
        x1 = samplex(N_samples=1)
        x2 = samplex(N_samples=1)
        y1 = sample_ya(x1)
        y2 = sample_yb(x2)
        print(y1.shape, y2.shape)
        diff_mmd = compare_samples_classifier_based(y1, y2, n_folds=2)

        res_list.append(diff_mmd)

    return pd.DataFrame(res_list)

posterior_y1y2_res = sample_posterior(n_x_samples = 100, sample_ya = sampley1, sample_yb = generate_sample_y_t(0.5))
posterior_y1y3_res = sample_posterior(n_x_samples = 100, sample_ya = sampley1, sample_yb = generate_sample_y_t(0.0))

print(f"""
Posterior y1 y2: {posterior_y1y2_res.mean()}    
Posterior y1 y3: {posterior_y1y3_res.mean()}
      """)

print(wilcoxon(posterior_y1y2_res, posterior_y1y3_res))

  0%|          | 0/100 [00:00<?, ?it/s]

torch.Size([100, 1]) torch.Size([100, 1])


  1%|          | 1/100 [00:00<01:22,  1.20it/s]

torch.Size([100, 1]) torch.Size([100, 1])


  2%|▏         | 2/100 [00:01<01:00,  1.62it/s]

...

torch.Size([100, 1]) torch.Size([100, 1])


 99%|█████████▉| 99/100 [00:55<00:00,  2.02it/s]

torch.Size([100, 1]) torch.Size([100, 1])


100%|██████████| 100/100 [00:56<00:00,  1.78it/s]


Posterior y1 y2: CST_accuracy         0.85215
CST_roc_auc_score    0.85215
dtype: float64    
Posterior y1 y3: CST_accuracy         0.84045
CST_roc_auc_score    0.84045
dtype: float64
      
WilcoxonResult(statistic=array([2021. , 2006.5]), pvalue=array([0.15173056, 0.1375889 ]))


In [9]:
different_t_res = []
for t in tqdm(torch.linspace(0, t_max, 10)):
    posterior_y1yt_res = sample_posterior(n_x_samples = 100, sample_ya = sampley1, sample_yb = generate_sample_y_t(t))
    different_t_res.append(posterior_y1yt_res.mean())

print(different_t_res)

  0%|          | 0/10 [00:00<?, ?it/s]

torch.Size([100, 1]) torch.Size([100, 1])


torch.Size([100, 1]) torch.Size([100, 1])


torch.Size([100, 1]) torch.Size([100, 1])


torch.Size([100, 1]) torch.Size([100, 1])


...

torch.Size([100, 1]) torch.Size([100, 1])


torch.Size([100, 1]) torch.Size([100, 1])


torch.Size([100, 1]) torch.Size([100, 1])


 40%|████      | 4/10 [03:47<05:41, 56.88s/it]


KeyboardInterrupt: 

In [ ]:
def sample_joint(
    n_x_samples = 1000,
    sample_ya = sampley1,
    sample_yb = sampley2
):
    res_list = []

    for i in tqdm(list(range(n_x_samples))):
        x1 = samplex(N_samples=N_samples)
        x2 = samplex(N_samples=N_samples)

        y1 = sample_ya(x1)
        y2 = sample_yb(x2)

        xy1 = torch.cat([x1, y1], dim=1)
        xy2 = torch.cat([x2, y2], dim=1)

        mmd = compare_samples_classifier_based(xy1, xy2, n_folds=3)

        res_list.append(mmd)

    return pd.DataFrame(res_list)



joint_y1y2_res = sample_joint(n_x_samples=100, sample_ya=sampley1, sample_yb=generate_sample_y_t(0.5))
joint_y1y3_res = sample_joint(n_x_samples=100, sample_ya=sampley1, sample_yb=generate_sample_y_t(0.0))

print(f"""
Joint y1 y2: {joint_y1y2_res.mean()}
Joint y1 y3: {joint_y1y3_res.mean()}
      """)

print(wilcoxon(joint_y1y2_res, joint_y1y3_res))

 20%|█▉        | 197/1000 [00:10<00:42, 18.71it/s]

In [ ]:
different_t_res = []
for t in tqdm(torch.linspace(0, t_max, 10)):
    joint_y1yt_res = sample_joint(n_x_samples=100, sample_ya=sampley1, sample_yb=generate_sample_y_t(t))
    different_t_res.append(joint_y1yt_res.mean())

print(different_t_res)

  0%|          | 0/10 [00:00<?, ?it/s]